In [3]:
import sys

moduleLoc = "/Users/Lykke-AndersenLab/PycharmProjects/AnoThER-Seq"
if moduleLoc in sys.path: 
    print('woo')
else:
    sys.path.append(moduleLoc)
    import matplotlib.pyplot as plt
import tools
import pandas as pd
import matplotlib
import seaborn as sns

# Enable inline plotting
%matplotlib inline

In [18]:
#read tail files into a pd dataframe
import imp
imp.reload(tools)
folderLoc = "/Users/Lykke-AndersenLab/Dropbox/NoctData/"
controlName = "siLuc.tails"
KDname = "siNoct.tails"
WTname = "NoctWT.tails"
EAname = "NoctEA.tails"

control = tools.pdTailMaker(folderLoc+controlName)
knockdown = tools.pdTailMaker(folderLoc+KDname)
WTaddback = tools.pdTailMaker(folderLoc+WTname)
EAaddback = tools.pdTailMaker(folderLoc+EAname)
print('Dataframes Made')


Dataframes Made


In [20]:
control.head()

,Gene,3Loc,TailLength,TailSeq,Type
0,RNU1-28P-201|snRNA,0,0,,
1,RNU1-28P-201|snRNA,0,0,,
2,RNU1-28P-201|snRNA,0,0,,
3,RNU1-28P-201|snRNA,0,0,,
4,RNU1-28P-201|snRNA,0,0,,
